# Music Brain Project

Can we guess how you will react to music?


In the spring of 2020, me (Ted Lewitt), Ben Hahn and Jack Elliott partnered with the Brain and Creativity Institute at USC to use deep learning methods to predict patients response to instrumental music.

### The Dataset

36 Patients went into a MRI and listened to 5 minute long instrumental songs that were either meant to be uplifting or sad. The MRI recorded their brain activations and the patients used a sliding scale to represent how they felt while listening to the music.

### Our Hypothesis

We can use a recurrent nueral network to predict how a patient felt at a moment in time, based off the MRI data from that moment. 


In [ ]:
import pandas as pd
import numpy as np
import nibabel as nib
import helperFunctions as hf
import keras
from keras.models import Sequential
from keras.layers import Activation,Dense
from keras.layers.recurrent import LSTM
import nilearn
import os
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

## Preprocessing

**DATA TYPES**
The MRI data was actually a fMRI, so it was a series of MRI data, which itself was a 3D volumetric matrix of brain activations.
The slider data was a text file with timestamps and activations, on a scale of 0 to 127, with 127 being very happy and 0 being very sad.

### Preprocessing the Slider Data
The fMRI data was at 1 Hz but the slider data was at 30 Hz, so we had to down-sample the data.
We used the average of the 30 measurements taken as the measurement for the one second.
We also scaled the slider data to be on a scale of 0,1.

**Bucketing the Data & Switching to Classification**
We were having issues with regressing the data (see our conculsions section for more info) so we decided to switch to a classsification problem, transforming our slider data from a scalar to a class, either Happy or Sad.

### Preprocessing the MRI Data
We used the niLearn package to transform our 4D fMRI (time steps, 3 spatial dimensions) to a 2D time series (time steps,
features). The initial shape of the input tensor was (495,100,90,100) and we wanted it to be (495,#Features) so it could be fed into our Tensorflow LSTM.
NiLearn has a class called NiftiMasker that does exactly that, transforming our data into an (495,250000) shaped time series. (This number of features is incedibly high we know, see our conclusions for more).


In [ ]:
NUM_BUCKETS = 2 # or 4
def preprocess_slider_data(data_file):
    data = load_data()
    down_sampled_data = down_sample(data)
    bucket_data = bucket(data,NUM_BUCKETS)
    return bucket_data
def load_data(file):
def down_sample(array,NUM_BUCKETS):

In [ ]:
txt_files_dir = ""
mri_files_dir = ""

for count in range(1,40):
    gotlabel=False
    gotnii=False
    if count<10:
        count=str(0)+str(count)
    
    #First we preprocess the nii file
    niifile="sub-"+str(count)+"_sadln_filtered_func_200hpf_standard_aroma.nii"
    else:
        fullNii=os.path.join(DATA_PATH_FOR_NII,niifile)
        if os.path.isfile(fullNii):
            nii,sizeValue=hf.niiToTS(fullNii)
            #nii=preprocessing.scale(nii)
            gotnii=True
            
    #Next up is the corresponding label data
        labelfile="sub-"+str(count)+"_snl_l_enjoy_log.txt"
        fullfile=os.path.join(DATA_PATH_FOR_LABELS,labelfile)
        if os.path.isfile(fullfile):        
            label=np.array(hf.sliderPre(fullfile,3))
            gotlabel=True 
            
    if gotlabel and gotnii:
        nii_array.append(nii)
        label_array.append(label)

nii_array=np.array(nii_array)
label_array=np.array(label_array)
    

# Conclusion

Why it didnt work-> far too many features, poor understanding of what the nilearn was doing, no domain knowledge

## Future Steps

Use a CNN-LSTM